# Flash calculations

Flash, or more general phase separation calculations is the process of determining the equilibrium state of a system under conditions specified by thermodynamic state functions.
At equilibrium the available mass splits into one or multiple physical states like liquid or gas.
To specify the thermodynamic state, three state quantities are required:

- mass of present components/ chemical species (mole numbers, overall compositions, ...)
- state function related to work energy (pressure or volume)
- state function related to thermal energy (temperature, internal energy, enthalpy)

PorePy provides some functionality to determine the equilibrium state of fluid mixtures.
It does **not** cover the full range of possible thermodynamic calculations and state definitions.
Instead it focuses on the properties and state definitions relevant for thermal compositional flow in the subsurface as indicated above.
One core ability of PorePy's flash framework is the ability to solve large amounts of flash calculations very efficiently,
making it suitable for cell-wise applications in flow simulations.

This tutorial explains the flash API and presents example calculations using a compiled persistent-variable flash.
A familiarity with the [tutorial on fluid modeling](./fluid_modeling.ipynb) and on [cubic equations of state](./cubic_eos.ipynb) is of advantage.

### Table of Contents

1. [Usage of numba](#usage-of-numba)
2. [General flash API](#general-flash-api)
3. [Example: Persistent-variable flash](#example-persistent-variable-flash)
4. [Flash solvers](#flash-solvers)
5. [Example: Semi-smooth Newton solver](#example-a-semi-smooth-newton-solver)
6. [Conclusion](#conclusion)

## Usage of numba

Available flash implementions in PorePy rely heavily in [numba](https://numba.pydata.org/).<br>
What you need to know as a user is that 

1. The framework relies on just-in-time compilation.
2. Compilation can be disabled for debugging or smaller problems.

The flash framework has certain levels of abstractions which require some ahead-of-time compilation.
While this is not yet fully supported by numba, we achieve a similar effect with static signature typing, which forces compilation ahead of usage.<br>
*This leads to a certain overhead in execution time when importing the flash module for the first time, and when compiling the flash for a specific fluid.*<br>
It also requires an explicit compilation step before the flash calculations are available.

To disable compilation and just use the pure Python implementation, do the following:

In [1]:
import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"

By uncommenting this flag, numba compilation is disabled and you will run the module with pure Python code.

The first thing you will notice is that it is significantly slower, but the calculations will start right away.
This is especially handy if you want to solve only a few problems to get an idea of how the mixture behaves, or you need some concrete values (try it!), or simply debug.

## General flash API

The flash is essentially the solution of a challenging system of algebraic and nonlinear equations.
While the structure of the equations is always the same, individual terms depend on the present components, their physical parameters and the underlying equation of state.

Therefore, the flash is implemented as an object containing templates for those equations.
It populates them with the information provides by an instance of [Fluid](../src/porepy/compositional/base.py#706).

The generel flash is hence instantiated using some fluid object.

In [2]:
from inspect import getsource, signature

import porepy.compositional.flash as pf

print(signature(pf.AbstractFlash.__init__))

(self, fluid: 'pp.Fluid[pp.FluidComponent, pp.Phase]', params: 'Optional[dict]' = None) -> 'None'


Once the flash object is created, the interface for the flash is as follows:

In [3]:
print(getsource(pf.AbstractFlash.flash))

    @abc.abstractmethod
    def flash(
        self,
        specification: StateSpecType,
        z: Optional[Sequence[np.ndarray | pp.number]] = None,
        /,
        *,
        initial_state: Optional[pp.compositional.FluidProperties] = None,
        params: Optional[dict] = None,
        **kwargs,
    ) -> FlashResults:
        """Abstract method for performing a flash procedure.

        The equilibrium state must be defined in terms of compositions ``z`` and two
        state functions declared in ``specification``.
        One state must relate to pressure or volume. The other to energy.

        Parameters:
            specifications: Equilibrium specifications in terms of state functions.
            z: ``default=None``

                Overall fractions of mass per component.

                It is only optional for pure fluids (``z`` is implicitly assumed to be
                1). For fluid mixtures with multiple components it must be of length
                ``num_compo

The minimum we need are equilibrium `specifications` in terms of two state functions.
PorePy supports two types of state specifications:

- Isobaric specifications (pressure at equilibrium is given)
- Isochoric specifications (specific volume of the fluid at equilibrium is given)

In [4]:
print(getsource(pf.IsobaricSpecifications))

class IsobaricSpecifications(TypedDict):
    """Typed dictionary for isobaric equilibrium specifications.

    The pressure values are obligatory and one energy-related variable is required.

    """

    p: np.ndarray | pp.number
    """Pressure at equilibrium."""

    T: NotRequired[np.ndarray | pp.number]
    """Temperature at equilibrium."""

    h: NotRequired[np.ndarray | pp.number]
    """Specific fluid enthalpy at equilibrium."""



We see that isobaric specifications have two additional optional fields, temperature and specific fluid enthalpy.
One of them must be given additionally.
Providing pressure and enthalpy values invokes the ph-flash for example.

Additionally to the state functions, the mass must be provided in form of overall compositions `z` (fractions of total mass associated with a component in the fluid). <br>
If the fluid has only 1 component, this argument is not required and implicitly assumed to be one.

The `initial_state` argument can be used to provide an initial guess for the flash problem. Otherwise the flash object has to compute them. *Note that this is not a trivial task in thermodynamics and can contribute noticeably to the overall time spent to solve the problem*.

The return value is a [FlashResults](../src/porepy/compositional/flash/abstract_flash.py##116) data structure, which is an extension of [FluidProperties](../src/porepy/compositional/states.py#231).
Additionally to all fluid properties evaluated at equilibrium, it also contains some metrics of the performed flash, like convergence flags and number of iterations.
This structure can be used to feed values into a global flow system for example.

## Example: Persistent-variable flash

## Flash solvers

Flash solvers constitute an own group of functionality in `porepy.compositional.flash.solvers`.

In short, we can say the following:

1. They must be numba-compiled functions, or shallow wrappers for C-functions (`cfunc`).
2. They solve a nonlinear set of equations represented by a function `f(x)`, taking and returning a numpy array.
3. They have a specified signature for compatibility with the remaining framework.

The package provides a signature object which can be used for compilation.
If `numba.njit` succeeds in compiling the solver with this signature object, all is good.

In [5]:
print('args:')
for i, a in enumerate(pf.SOLVER_FUNCTION_SIGNATURE.args):
    print(f'  {i}: {a}')
print('return type:')
print(pf.SOLVER_FUNCTION_SIGNATURE.return_type)

args:
  0: array(float64, 1d, A)
  1: FunctionType[array(float64, 1d, A)(array(float64, 1d, A))]
  2: FunctionType[array(float64, 2d, A)(array(float64, 1d, A))]
  3: DictType[unicode_type,float64]<iv=None>
  4: IntEnum<int64>(FlashSpec)
return type:
Tuple(array(float64, 1d, A), int64, int64)


A flash solver takes the following arguments:

0. An initial guess `X0`.
1. A function representing the residual of the nonlinear equations `f(x)`.
2. A function representing the Jacobian of `f`.
3. A parameter dictionary with `str`-`float` pairs, containing any parameters the solver might need.
4. The Flash specification using `porepy.compositional.flash.FlashSpec`.

It returns:

0. The solution `X_n` satisfying `f(X_n) == 0`.
1. An exit code.
2. The number of performed iterations.

The following exit codes are reserved:

- 0: Converged
- 1: Maximal number of iterations reached
- 2: Divergence (`nan` or `infty` detected)
- 3: Failure in evaluating `f`
- 4: Failure in evaluating Jacobian of `f`
- 5: Any other failure.

This and more information, developers can find in the core module `porepy.compositional.flash.solvers._core`.

This signature enables us to easily serialize or parallelize large quantities of flash problems.
The so-called multi-solvers are like base solvers, with the exception that they take the solver itself to be applied to individual problems.
They are bundled in the solver package as follows.

In [6]:
from porepy.compositional.flash.solvers._core import _multi_solver_signature

for k, v in pf.MULTI_SOLVERS.items():
    print(f'Solver key: {k}')
    print(f'Solver function: {v}')

print('args:')
for i, a in enumerate(_multi_solver_signature.args):
    print(f'  {i}: {a}')
print('return type:')
print(_multi_solver_signature.return_type)


Solver key: sequential
Solver function: CPUDispatcher(<function sequential_solver at 0x000001FA0E7184A0>)
Solver key: parallel
Solver function: CPUDispatcher(<function parallel_solver at 0x000001FA0E746F20>)
args:
  0: array(float64, 2d, A)
  1: FunctionType[array(float64, 1d, A)(array(float64, 1d, A))]
  2: FunctionType[array(float64, 2d, A)(array(float64, 1d, A))]
  3: FunctionType[Tuple(array(float64, 1d, A), int64, int64)(array(float64, 1d, A), FunctionType[array(float64, 1d, A)(array(float64, 1d, A))], FunctionType[array(float64, 2d, A)(array(float64, 1d, A))], DictType[unicode_type,float64]<iv=None>, IntEnum<int64>(FlashSpec))]
  4: DictType[unicode_type,float64]<iv=None>
  5: IntEnum<int64>(FlashSpec)
return type:
Tuple(array(float64, 2d, C), array(int64, 1d, C), array(int64, 1d, C))


We see that the arguments and return values of the multi-solvers are analogous to the base solvers, with the exception that they take additionally the base solver as the fourth (3.) argument

### Generic Flash Arguments

Flash systems are as mentioned above represented as functions `f(x)` to simplify the solver interface.
In reality, flash systems require also the state functions which are not degrees of freedom as arguments, and potentially some parameters.

All that is formalized in something called *generic flash argument*.
Essentially, it is a pattern to put state specifications, parameters and actual degrees of freedom into a 1D array, which is readable by `f` and its Jacobian.

There is a whole module concergned with the assembly and the parsing of this generic flash arguments.
If you develop a solver or a flash for a new flash specification, you have to familiarize yourself with them in [flash_equations.py](../src/porepy/compositional/flash/flash_equations.py)

## Example: A semi-smooth Newton solver

In this section we build a new type of flash solver and demonstrate how to register them it with the framework, and subsequently use it.

The solver built here is a semi-smooth Newton solver, which uses the `min` function for the complementarity conditions.

There are some things we need to know before attempting to do that:

1. The structure of any flash residual function is such that the last `num_phases` entries are the complementarity conditions per phase.
2. The Jacobians are such that the columns always correspond to a specific degree of freedom. The last `num_phase * num_components` entries are for the partial fractions. The `num_phase - 1` entries before them are for the independent phase fractions.
3. DOFs of the first phase (phase fraction and saturation) are always eliminated by unity of fractions.

In [7]:
import numpy as np
from typing import Callable

from porepy.compositional._core import njit


@njit(pf.SOLVER_FUNCTION_SIGNATURE)
def semismooth_newton(
    X0: np.ndarray,
    F: Callable[[np.ndarray], np.ndarray],
    DF: Callable[[np.ndarray], np.ndarray],
    params: dict,
    spec: pf.FlashSpec,
) -> tuple[np.ndarray, int, int]:
    # Parsing solver parameters which are always passed by the PersistentVariableFlash.
    ncomp = int(params["num_components"])
    nphase = int(params["num_phases"])
    f_dim = int(params["f_dim"])
    tol = float(params["tolerance"])
    max_iter = int(params["max_iterations"])

    # Default return values: no convergence.
    num_iter = 0
    exitcode = 1

    # Creating update vector.
    X = X0.copy()
    DX = np.zeros_like(X0)

    # NOTE: This can of course be done more efficiently without assembling the
    # smooth complementarity conditions and their Jacobian fully.
    def F_ss(_X: np.ndarray) -> np.ndarray:
        F_val = F(_X)
        gen = pf.parse_generic_arg(_X, ncomp, nphase, spec)
        # Parsing partial fractions and phase fractions.
        x = gen[1]
        y = gen[2]
        unity = 1.0 - x.sum(axis=1)

        # Last nphase entries are always the complementarity conditions.
        for j in range(nphase):
            if y[j] <= unity[j]:
                F_val[-nphase + j] = y[j]
            else:
                F_val[-nphase + j] = unity[j]

        return F_val

    def DF_ss(_X: np.ndarray) -> np.ndarray:
        DF_val = DF(_X)
        gen = pf.parse_generic_arg(_X, ncomp, nphase, spec)
        # Parsing partial fractions and phase fractions.
        x = gen[1]
        y = gen[2]
        unity = 1.0 - x.sum(axis=1)
        npnc = ncomp * nphase

        # New block to be inserted into the Jacobian for the complementarity conditions.
        new_block = np.zeros((nphase, DF_val.shape[1]))
        for j in range(nphase):
            if y[j] <= unity[j]:
                new_block[1:, -(npnc + nphase - 1) : -npnc] = np.eye(nphase - 1)
                # First phase is always eliminated by unity.
                new_block[0, -(npnc + nphase - 1) : -npnc] = -1.0
            else:
                new_block[j, -(npnc + j * ncomp) : -(npnc + (j + 1) * ncomp)] = -1.0

        DF_val[-nphase:, :] = new_block
        return DF_val

    try:
        f_i = F_ss(X)
    except Exception:  # Faulty residual assembly.
        return X, 3, num_iter

    res_i = np.linalg.norm(f_i)

    if res_i <= tol:
        exitcode = 0  # root already found
    else:
        for _ in range(max_iter):
            num_iter += 1

            # Divergence check.
            if (
                np.any(np.isnan(X))
                or np.any(np.isinf(X))
                or np.any(np.isnan(f_i))
                or np.any(np.isinf(f_i))
            ):
                exitcode = 2
                break

            try:
                df_i = DF_ss(X)
            except Exception:  # Faulty Jacobian assembly.
                exitcode = 4
                break

            try:
                DX[-f_dim:] = np.linalg.solve(df_i, -f_i)
            except Exception:  # Failure in linear solver.
                exitcode = 5
                break

            # Divergence check.
            if np.any(np.isnan(DX)) or np.any(np.isinf(DX)):
                exitcode = 2
                break

            X = X + DX

            try:
                f_i = F_ss(X)
            except Exception:
                exitcode = 3
                break

            # Convergence check.
            res_i = np.linalg.norm(f_i)
            if res_i <= tol:
                exitcode = 0
                break

    return X, exitcode, num_iter

We now have a new solver which we want to use.
We register it by adding it to the solver framework.

In [8]:
pf.SOLVERS['semismooth_newton'] = semismooth_newton

Now let's attempt to solve one of the previous problems with the new solver and let's see how it performs.

In [9]:
flash1: pf.CompiledPersistentVariableFlash = ...
results1: pf.FlashResults = ...

results_semismooth = flash1.flash(
    {
        'p': ...,
        'h': ...,
    },
    [0.99, 0.01],
    params={
        "solver": "semismooth_newton",
        "mode": "parallel",
    },
)

print(results1.converged.sum())
print(results_semismooth.converged.sum())

AttributeError: 'ellipsis' object has no attribute 'flash'

As expected, the default solver which uses an interior-point approach (and a couple of other tricks) performs overall better then a simple semi-smooth Newton.

## Conclusion